# Code to mount Google Drive

In [54]:
# mount google drive to colab
from google.colab import drive
import os

n = 140
drive.mount('/content/drive')
print("#" * n)

# change working directory
print("### colabs default working directory")

print("###\n### path: {}".format(os.getcwd()))
print("#" * n)
print("### setted working directory")

%cd drive/Shareddrives/MADS_Milestone2/


print("###\n### {}".format(os.getcwd()))
print("#" * n)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
############################################################################################################################################
### colabs default working directory
###
### path: /content/drive/Shareddrives/MADS_Milestone2
############################################################################################################################################
### setted working directory
[Errno 2] No such file or directory: 'drive/Shareddrives/MADS_Milestone2/'
/content/drive/Shareddrives/MADS_Milestone2
###
### /content/drive/Shareddrives/MADS_Milestone2
############################################################################################################################################


# Import libraries

In [55]:
from platform import python_version
import sys
import pandas as pd
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

print("used python version {}".format(python_version()))

used python version 3.7.12


## import all final input files to integrate the different network layers


In [56]:
# current working directory
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"

# get all the file names of data
files = os.listdir(DIR+"/res")

print("#"*140, "\n### ", files)

###########################################
# define abbreviations of diferent datasets
# c_  = curated
# gda = gene - disease association
# vda = variant - disease association
# dda= disease - disease association
###########################################

final_files = [s for s in files if "INTEGRATED" in s]
names = ["base", "advanced"]

DF = {n :pd.read_csv("{}/res/{}".format(DIR, f)) for f,n in zip(final_files,names)}
DF

############################################################################################################################################ 
###  ['processed_protein_protein_interactions.csv', 'final_protein_protein_interaction_network.csv', 'final_disease_protein_interaction_network.csv', 'final_drug_protein_interaction_network.csv', 'intermediate_biological_function.csv', 'final_bf_bf_and_bf_protein_interaction_network.csv', 'INTEGRATED_FINAL_NETWORK.csv', 'INTEGRATED_FINAL_NETWORK_with_bf.csv', 'filtered_base_network.csv', 'intermediate_protein_biological_function_map.csv', 'keep_biological_functions.csv', 'filtered_base_incl_bf_layer_network.csv', 'keep_biological_functions_network2_one_approval.csv', 'filtered_base_network2_one_approval.csv', 'filtered_base_incl_bf_layer_network2_one_approval.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,12,13,14,16,19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,1,12,13,14,16,19,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


{'advanced':              node1       node2  ... hasBeenWithdrawn             edge_type
 0              381         375  ...              NaN                   NaN
 1              381        9727  ...              NaN                   NaN
 2              381        2108  ...              NaN                   NaN
 3              381       23647  ...              NaN                   NaN
 4             4074       26088  ...              NaN                   NaN
 ...            ...         ...  ...              ...                   ...
 329238  GO:2001284  GO:0038055  ...              NaN             regulates
 329239  GO:2001285  GO:0038055  ...              NaN  negatively_regulates
 329240  GO:2001286  GO:0072584  ...              NaN             regulates
 329241  GO:2001287  GO:0072584  ...              NaN  negatively_regulates
 329242  GO:2001288  GO:0072584  ...              NaN  positively_regulates
 
 [329243 rows x 24 columns],
 'base':                 node1  node2  ... wi

In [57]:
base = DF["base"].copy()
advanced = DF["advanced"].copy()

# remove the nodes which are still in  format of ensemble gene ids
rem = [False if "ENSG" in str(r["node1"]) or "ENSG" in str(r["node2"]) else True  for i,r in base.iterrows()]
base = base[rem]

rem = [False if "ENSG" in str(r["node1"]) or "ENSG" in str(r["node2"]) else True  for i,r in advanced.iterrows()]
advanced = advanced[rem]

# make sure to have correct format for protein nodes
base.node1 = [str(int(float(r["node1"]))) if r["node1_type_orig"] == "protein" else r["node1"] for i,r in base.iterrows() ]
base.node2 = [str(int(float(r["node2"]))) if r["node2_type_orig"] == "protein" else r["node2"] for i,r in base.iterrows() ]

advanced.node1 = [str(int(float(r["node1"]))) if r["node1_type_orig"] == "protein" else r["node1"] for i,r in advanced.iterrows() ]
advanced.node2 = [str(int(float(r["node2"]))) if r["node2_type_orig"] == "protein" else r["node2"] for i,r in advanced.iterrows() ]

len(base), len(advanced)

(205428, 329091)

# potential filter criteria
- remove all drugs which are no inhibitors
- remove indication nodes which are redudant or too unspecific
- remove nodes which are isolated



In [58]:
base_org = len(base)
advanced_org = len(advanced)


print("### base network")

# remove all drugs that are no specific inhibitors - keep only INHIBITOR, ANTAGONIST, BLOCKER, NEGATIVE ALLOSTERIC MODULATOR, INVERSE AGONIST, NEGATIVE MODULATOR, DEGRADER, ANTISENSE INHIBITOR  
rem = ["AGONIST", "POSITIVE ALLOSTERIC MODULATOR", "MODULATOR", "BINDING AGENT", "POSITIVE MODULATOR", "STABILISER", "PARTIAL AGONIST", "ACTIVATOR", "HYDROLYTIC ENZYME", "OPENER", "DISRUPTING AGENT", "CROSS-LINKING AGENT", "OTHER", "RELEASING AGENT", "SUBSTRATE", "PROTEOLYTIC ENZYME"]
base = base[~base.isin(rem).ActionType_drug__filter]
print("drug modality removed {} drugs which are not inhibiting the target".format(base_org-len(base)))
base_rem = len(base)

# remove drugs which did not reach the market yet
base = base[~(base.maximumClinicalTrialPhase < 4)]
print("drugs that did not went through all clinical phases removed {} drugs".format(base_rem-len(base)))
base_rem = len(base)

# remove drugs which has been withdrawn
base = base[~(base.hasBeenWithdrawn == 1)]
print("drugs which has been withdrawn removed {} drugs".format(base_rem-len(base)))

print("### advanced network")

advanced = advanced[~advanced.isin(rem).ActionType_drug__filter]
print("drug modality removed {} drugs which are not inhibiting the target".format(advanced_org-len(advanced)))
advanced_rem = len(advanced)

# remove drugs which did not reach the market yet
advanced = advanced[~(advanced.maximumClinicalTrialPhase < 4)]
print("drugs that did not went through all clinical phases removed {} drugs".format(advanced_rem-len(advanced)))
advanced_rem = len(advanced)

# remove drugs which has been withdrawn
advanced = advanced[~(advanced.hasBeenWithdrawn == 1)]
print("drugs which has been withdrawn removed {} drugs".format(advanced_rem-len(advanced)))

print("#"*100)
print("in total we have removed {} drugs from base network\nand {} from the advanced network".format(base_org-len(base),advanced_org-len(advanced)))
len(base), len(advanced)

### base network
drug modality removed 3921 drugs which are not inhibiting the target
drugs that did not went through all clinical phases removed 4069 drugs
drugs which has been withdrawn removed 333 drugs
### advanced network
drug modality removed 3921 drugs which are not inhibiting the target
drugs that did not went through all clinical phases removed 4069 drugs
drugs which has been withdrawn removed 333 drugs
####################################################################################################
in total we have removed 8323 drugs from base network
and 8323 from the advanced network


(197105, 320768)

In [59]:
indications = set(base.node2_name[base.node2_type=="disease"])
print("{} unique indications".format(len(indications)))
base_org = len(base)
advanced_org = len(advanced)

rem = [i for i in list(indications)  if "disease" in i]
base = base[~base.isin(rem).node2_name]
advanced = advanced[~advanced.isin(rem).node2_name]
                    
print("generic indications with 'disease' in name removed {} indicatiobs".format(base_org-len(base)))
base_rem = len(base)
print("generic indications with 'disease' in name removed {} indicatiobs".format(advanced_org-len(advanced)))
advanced_rem = len(advanced)
                    


drug_approvals = pd.read_pickle(DIR + 'dat/dfs/indication')

disease_mapping = pd.read_csv(DIR + "/dat/disease_mappings.tsv.gz", compression='gzip',  sep='\t') 

disease_mapping["convertion"] = disease_mapping.apply(lambda x:  x["vocabulary"] + "_" + x["code"]  , axis=1) 
disease_mapping = disease_mapping[disease_mapping.vocabulary == "EFO"]
disease_mapping[disease_mapping.convertion == "EFO_0000538"]

disease_mapping

9347 unique indications
generic indications with 'disease' in name removed 297 indicatiobs
generic indications with 'disease' in name removed 297 indicatiobs


,diseaseId,name,vocabulary,code,vocabularyName,convertion
13636,C4721444,Burkitt Leukemia,EFO,0000094,B-cell acute lymphoblastic leukemia,EFO_0000094
13637,C0862030,Precursor B-lymphoblastic lymphoma/leukemia,EFO,0000094,B-cell acute lymphoblastic leukemia,EFO_0000094
13638,C0023485,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,EFO,0000094,B-cell acute lymphoblastic leukemia,EFO_0000094
13639,C1292769,Precursor B-cell lymphoblastic leukemia,EFO,0000094,B-cell acute lymphoblastic leukemia,EFO_0000094
13640,C0855095,Small Lymphocytic Lymphoma,EFO,0000095,chronic lymphocytic leukemia,EFO_0000095
...,...,...,...,...,...,...
18673,C0268731,Renal glomerular disease,EFO,1002049,glomerular disease,EFO_1002049
18674,C0268732,Nephritic syndrome,EFO,1002050,nephritis,EFO_1002050
18675,C0149937,Acute interstitial nephritis,EFO,1002050,nephritis,EFO_1002050
18676,C0027697,Nephritis,EFO,1002050,nephritis,EFO_1002050


In [60]:
drugs = list(set(base[base.node1_type =="drug"].node1))

indications = list(set(base[base.node2_type =="disease"].node2))


drug_approvals["filter"] = False
i = 0
for r in drug_approvals.id:
  if r in drugs:
    drug_approvals["filter"].iloc[i] = True
  i += 1

drug_approvals["approvalCount"] = [len(i) for i in drug_approvals.approvedIndications]
display(drug_approvals)
drug_approvals[drug_approvals["filter"]].value_counts("approvalCount") 



/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,id,indications,approvedIndications,indicationCount,filter,approvalCount
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...","[EFO_0003966, EFO_0000540]",120,True,2
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...","[EFO_0000616, EFO_0000339, EFO_0000311]",117,True,3
2,CHEMBL1172928,"[{'disease': 'MONDO_0002009', 'efoName': 'majo...",[],1,False,0
3,CHEMBL1234886,"[{'disease': 'EFO_0003103', 'efoName': 'urinar...",[],69,False,0
4,CHEMBL3039596,"[{'disease': 'EFO_0000537', 'efoName': 'hypert...",[EFO_0000537],1,True,1
...,...,...,...,...,...,...
7986,CHEMBL2108209,"[{'disease': 'EFO_0000764', 'efoName': 'hiv in...",[],1,False,0
7987,CHEMBL2108029,"[{'disease': 'MONDO_0007254', 'efoName': 'brea...",[],2,False,0
7988,CHEMBL3989777,"[{'disease': 'MONDO_0003005', 'efoName': 'macu...",[],1,False,0
7989,CHEMBL1255794,"[{'disease': 'EFO_0000764', 'efoName': 'hiv in...",[],1,False,0


approvalCount
1     405
2     283
3     141
0     126
4      77
5      57
6      28
7      14
9       8
8       5
10      4
12      2
14      2
17      1
11      1
13      1
15      1
18      1
dtype: int64

In [61]:
# threshold - numbver of approved drugs for each indications
thr = 60
# count number of approved drugs for each indication
test = drug_approvals.explode("approvedIndications").groupby("approvedIndications").count()

# filter the indications out with too many durg approvals - are likely to be too generic like neoplasm cancer and so on and maybe also uninteresting if there are too many drug approvals
display(disease_mapping[disease_mapping.isin(list(test[test.id>thr].index)).convertion])


rem = disease_mapping[disease_mapping.isin(list(test[test.id>60].index)).convertion].diseaseId.tolist()

base = base[~base.isin(rem).node2]
advanced = advanced[~advanced.isin(rem).node2]
                    
print("indications with drug approvals higher than {} removed {} indications".format(thr, base_rem-len(base)))
base_rem = len(base)
print("indications with drug approvals higher than {} removed {} indications".format(thr, advanced_rem-len(advanced)))
advanced_rem = len(advanced)

,diseaseId,name,vocabulary,code,vocabularyName,convertion
13835,C0006826,Malignant Neoplasms,EFO,0000311,cancer,EFO_0000311
13836,C0347071,Malignant neoplasm of other and unspecified sites,EFO,0000311,cancer,EFO_0000311
13844,C0007222,Cardiovascular Diseases,EFO,0000319,cardiovascular disease,EFO_0000319
13845,C0728936,Disorder of circulatory system,EFO,0000319,cardiovascular disease,EFO_0000319
13992,C0155616,Secondary hypertension,EFO,0000537,hypertension,EFO_0000537
13993,C0085580,Essential Hypertension,EFO,0000537,hypertension,EFO_0000537
13994,C0020538,Hypertensive disease,EFO,0000537,hypertension,EFO_0000537
14020,C0004364,Autoimmune Diseases,EFO,0000540,immune system disease,EFO_0000540
14021,C0021053,Immune System Diseases,EFO,0000540,immune system disease,EFO_0000540
14022,C3714514,Infection,EFO,0000544,infection,EFO_0000544


indications with drug approvals higher than 60 removed 656 indications
indications with drug approvals higher than 60 removed 656 indications


In [70]:
test.sort_values("id", ascending=False)

,id,indications,indicationCount,filter,approvalCount
approvedIndications,,,,,
EFO_0000616,245,245,245,245,245
EFO_0000319,184,184,184,184,184
EFO_0000771,163,163,163,163,163
EFO_0000311,135,135,135,135,135
EFO_0003843,105,105,105,105,105
...,...,...,...,...,...
EFO_0000558,1,1,1,1,1
EFO_0000557,1,1,1,1,1
EFO_0009492,1,1,1,1,1


In [69]:
disease_mapping[disease_mapping.isin(list(test.sort_values("id", ascending=False).index.tolist()[:10])).convertion] 

,diseaseId,name,vocabulary,code,vocabularyName,convertion
13835,C0006826,Malignant Neoplasms,EFO,0000311,cancer,EFO_0000311
13836,C0347071,Malignant neoplasm of other and unspecified sites,EFO,0000311,cancer,EFO_0000311
13844,C0007222,Cardiovascular Diseases,EFO,0000319,cardiovascular disease,EFO_0000319
13845,C0728936,Disorder of circulatory system,EFO,0000319,cardiovascular disease,EFO_0000319
13992,C0155616,Secondary hypertension,EFO,0000537,hypertension,EFO_0000537
13993,C0085580,Essential Hypertension,EFO,0000537,hypertension,EFO_0000537
13994,C0020538,Hypertensive disease,EFO,0000537,hypertension,EFO_0000537
14022,C3714514,Infection,EFO,0000544,infection,EFO_0000544
14081,C0027651,Neoplasms,EFO,0000616,neoplasm,EFO_0000616
14251,C0003723,Arbovirus Infections,EFO,0000763,viral disease,EFO_0000763


In [ ]:
indic = drug_approvals[drug_approvals["filter"]].explode("approvedIndications").drop_duplicates("approvedIndications").approvedIndications.tolist().copy()
indic = disease_mapping[disease_mapping.isin(indic).convertion].diseaseId.tolist()

filt = []
for i,r in base.iterrows():
  if r["node2_type"] == "disease":
    if r["node2"] not in indic:
      filt += [False]
    else:
      filt += [True]
  else:
      filt += [True]


In [ ]:
base = base[filt]


In [ ]:

filt = []
for i,r in advanced.iterrows():
  if r["node2_type"] == "disease":
    if r["node2"] not in indic:
      filt += [False]
    else:
      filt += [True]
  else:
      filt += [True]


In [ ]:
advanced = advanced[filt]

In [ ]:
len(list(set(base[base.node2_type =="disease"].node2)))

dis = list(set([r["node2_name"] for i,r  in base.iterrows() if r["node2_type"] == "disease"]))
dis

['Melanoma astrocytoma syndrome',
 'Alopecia, Male Pattern',
 'Malignant tumor of colon',
 'Mixed Cellularity Hodgkin Lymphoma',
 'Hereditary Papillary Renal Carcinoma',
 'Colon Carcinoma',
 'Hamman-Rich Disease',
 'Leishmaniasis, Cutaneous',
 'Gastric ulcer',
 'Acute lymphocytic leukemia',
 'Cerebral Infarction',
 'Malignant neoplasm of prostate',
 'Chorea, Benign Familial',
 'Renal carcinoma',
 'Chromophobe Renal Cell Carcinoma',
 'Chronic Obstructive Airway Disease',
 'AUTISM, SUSCEPTIBILITY TO, X-LINKED 6',
 'ASTHMA-RELATED TRAITS, SUSCEPTIBILITY TO, 5',
 'Manic Disorder',
 'Malignant lymphoma - lymphocytic, intermediate differentiation',
 'Carcinoma, Ovarian Epithelial',
 'Acute Promyelocytic Leukemia',
 'THROMBOCYTHEMIA 1',
 'PARKINSON DISEASE 22, AUTOSOMAL DOMINANT',
 'Sinusitis',
 'DIABETES MELLITUS, NONINSULIN-DEPENDENT, 5',
 'Hay fever',
 'Deep Vein Thrombosis',
 'THROMBOCYTHEMIA 3',
 'Benign Prostatic Hyperplasia',
 'PARKINSON DISEASE, LATE-ONSET',
 'PAGET DISEASE OF BONE 2,

In order to allow the model to focus on the biological functions most relevant to treatment, we only consider biological functions which are associated with at least one drug target or one disease protein, either directly or implicitly through their children.

In [ ]:
evidence_code = ["EXP", # Inferred from Experiment (EXP)
"IDA",  # Inferred from Direct Assay (IDA)
# "IPI",  # Inferred from Physical Interaction (IPI)
"IMP",  # Inferred from Mutant Phenotype (IMP)
"IGI",  # Inferred from Genetic Interaction (IGI)
# "IEP",  # Inferred from Expression Pattern (IEP)
# Each of the experimental evidence codes have a corresponding ‘high throughput’ evidence code (HTP). HTP are a type of experimental evidence that indicate that the annotation is supported by high throughput methodologies. The high throughput evidence codes are:

"HTP",  # Inferred from High Throughput Experiment (HTP)
"HDA",  # Inferred from High Throughput Direct Assay (HDA)
"HMP",  # Inferred from High Throughput Mutant Phenotype (HMP)
"HGI"  # Inferred from High Throughput Genetic Interaction (HGI)
# "HEP",  # Inferred from High Throughput Expression Pattern (HEP)
]

In [ ]:
# filt = [True if (r["node1_type"] == "biological_process") and (r["node2_type_orig"] == "protein") else False for i,r in advanced.iterrows()]

# advanced = advanced[list(~np.array(filt))]

In [ ]:
go_treat = list(set([r["node1"] for i,r in advanced.iterrows() if ((r["node2_type"] == "disease_related_protein") or (r["node2_type"] == "drug_target")) and (r["node1_type"] == "biological_process")]))
len(go_treat)

11476

In [ ]:
go = list(set([r["node1"] for i,r in advanced.iterrows() if r["node1_type"] == "biological_process"]))
go += list(set([r["node2"] for i,r in advanced.iterrows() if r["node2_type"] == "biological_process"]))

go = list(set(go))

# import mygene library to connect to the API
try:
    import mygene
except ModuleNotFoundError as err:
    !pip install mygene
    import mygene
    # Error handling
    print(err)
    
# make API connection
mg = mygene.MyGeneInfo()

# fetch all genes for each biological function
out = mg.querymany(go, scopes='GO', fields='entrezgene,symbol,go.BP.evidence,go.BP.id', species='human', returnall=True)


querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-28503...done.
Finished.
9114 input query terms found dup hits:
	[('GO:0048790', 6), ('GO:0060018', 4), ('GO:0046882', 3), ('GO:0042073', 10), ('GO:0051098', 4), ('G
16024 input query terms found 

In [ ]:
bp = pd.DataFrame()

for i in out["out"]:

  new = pd.DataFrame([i])
  new_col = list(set(list(new.columns)) - set(bp.columns))

  bp[new_col] = None
  bp = pd.concat([bp,new])

bp.reset_index(drop=True).to_csv(DIR + "res/intermediate_protein_biological_function_map.csv")

bp

,notfound,query,go,symbol,entrezgene,_score,_id
0,True,GO:1903111,None,None,None,NaN,None
0,NaN,GO:1990705,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0000122'...",NOTCH2,4853,23.146150,4853
0,True,GO:0052200,NaN,NaN,NaN,NaN,NaN
0,True,GO:1902875,NaN,NaN,NaN,NaN,NaN
0,True,GO:1903857,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
0,True,GO:1903227,NaN,NaN,NaN,NaN,NaN
0,NaN,GO:0014044,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0007411'...",LAMB2,3913,19.180246,3913
0,NaN,GO:0014044,"{'BP': [{'evidence': 'ISS', 'id': 'GO:0000165'...",NF1,4763,19.180246,4763
0,NaN,GO:0014044,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0001843'...",MED12,9968,19.180246,9968


In [ ]:
from ast import literal_eval

def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s
bp = pd.read_csv(DIR + "res/intermediate_protein_biological_function_map.csv", dtype= {"entrezgene": str})
bp.go = bp.loc[:,["go"]].applymap(try_literal_eval)
bp

,Unnamed: 0,notfound,query,go,symbol,entrezgene,_score,_id
0,0,True,GO:1903111,NaN,NaN,NaN,NaN,NaN
1,1,NaN,GO:1990705,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0000122'...",NOTCH2,4853,23.146150,4853.0
2,2,True,GO:0052200,NaN,NaN,NaN,NaN,NaN
3,3,True,GO:1902875,NaN,NaN,NaN,NaN,NaN
4,4,True,GO:1903857,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
73386,73386,True,GO:1903227,NaN,NaN,NaN,NaN,NaN
73387,73387,NaN,GO:0014044,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0007411'...",LAMB2,3913,19.180246,3913.0
73388,73388,NaN,GO:0014044,"{'BP': [{'evidence': 'ISS', 'id': 'GO:0000165'...",NF1,4763,19.180246,4763.0
73389,73389,NaN,GO:0014044,"{'BP': [{'evidence': 'IEA', 'id': 'GO:0001843'...",MED12,9968,19.180246,9968.0


In [ ]:
keep = []
rem = []
ind = []
bp.reset_index(drop=True, inplace=True)
n = 0
for i,r in bp.iterrows():
  if r["notfound"] == True:
    keep += [(r["query"], r["entrezgene"])]
    rem += [True] 
    ind += [i]
  else:
  
    if isinstance(r["go"]["BP"], list):
      BP = r["go"]["BP"]
    else:
      # print(n)
      BP = [r["go"]["BP"]]
      # print(BP)

    for j in BP:

      if (j["evidence"] in evidence_code) and (n == 0):
        keep += [(r["query"], r["entrezgene"])]
        rem += [False]
        ind += [i]
        n += 1
   
    if ind[-1] != i:      
        keep += [(r["query"], r["entrezgene"])]
        rem += [True]
        ind += [i]

  
      
    n = 0 

len(keep), len(rem), bp.shape

(73391, 73391, (73391, 8))

In [ ]:
bp_keep = bp[~np.array(rem)].copy()
KEEP = [g+ "_" + str(e) for e,g in zip(bp_keep.entrezgene.to_list(), bp_keep["query"].to_list())]

In [ ]:
 filt = [True if (r["node1_type"] == "biological_process") and (r["node2_type_orig"] == "protein") else False for i,r in advanced.iterrows()]

 adv = [g+ "_" + e for e,g in zip(advanced[filt].node2.to_list(), advanced[filt]["node1"].to_list())]
 print(len(set(KEEP)), len(set(adv)), len(set(KEEP) - set(adv )),len(set(adv )-set(KEEP)))
 print(len(set(KEEP).intersection(set(adv))))

47615 44736 9940 7061
37675


In [ ]:
GO = [i.split("_")[0] for i in list(set(KEEP).intersection(set(adv)))]
ent = [i.split("_")[1] for i in list(set(KEEP).intersection(set(adv)))]

keep = advanced[(pd.concat([advanced.isin(GO).loc[:,["node1"]], advanced.isin(ent).loc[:,["node2"]]], axis=1).sum(axis=1) > 1)]


advanced = advanced[~np.array(filt)].append(keep).reset_index(drop=True)

# remove all biological functions which do not have at least one drug or disease node

In [ ]:
# # %%timeit
go_treat =  list(set([r["node1"] for i,r in advanced.iterrows() if ((r["node2_type"] == "disease_related_protein") or (r["node2_type"] == "drug_target")) and (r["node1_type"] == "biological_process")])) #['GO:0019918', "GO:0021516"] #,
go = go_treat

n = 0
while True:
  ind = advanced.isin(go_treat).loc[:,["node1"]]
  ind = advanced[ind["node1"]].index
 
  df = advanced.loc[ind]
  
  if df.empty:
    print("finish")
    print(n)
    break
  
  ind = list(~np.array(df.isin(["protein"]).loc[:, "node2_type_orig"].tolist()))

  df = df[ind]
  if df.empty:
    print("finish")
    print(n)
    break
    
  go_treat = df.node2.tolist() 
  go += go_treat
  n += 1

go = list(set(go))
GO = pd.DataFrame (go, columns = ['keep_GO'])
GO.to_csv(DIR + "res/keep_biological_functions.csv")

finish
19


In [ ]:
ind = advanced.isin(go).loc[:,["node1", "node2"]]
ix = advanced[ind.loc[:,"node1"]]
iy = advanced[ind.loc[:,"node2"]]
ib = [True if ((r.node1_type != "biological_process") and (r.node2_type != "biological_process")) else False for i,r in advanced.iterrows()]


In [ ]:
print("shape before filtering of biological function layer")
print(advanced.shape)
print("shape before filtering of biological function layer considering only base layers")
print(advanced[ib].shape)

advanced = ix.append(iy).reset_index().drop_duplicates("index").set_index("index").append(advanced[ib])
print("after filtering of biological function layer")
print(advanced.shape)

shape before filtering of biological function layer
(262572, 24)
shape before filtering of biological function layer considering only base layers
(151559, 24)
after filtering of biological function layer
(249823, 24)


In [ ]:
base.to_csv(DIR + "res/filtered_base_network.csv")
advanced.to_csv(DIR + "res/filtered_base_incl_bf_layer_network.csv")

In [ ]:
advanced.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(advanced.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")



node_type
biological_process         25945
disease_related_protein     7360
protein                     5741
drug_target                 1378
drug                        1339
disease                      365
dtype: int64

In [ ]:
# # go_treat =  ['GO:0019918', "GO:0021516"] #,
# go = go_treat

# n = 0
# while True:
#   ind = advanced.isin(go_treat).loc[:,["node1"]]
#   ind = advanced[ind["node1"]].index
 
#   df = advanced.loc[ind]
#   # ind = list(~np.array(df.isin(["protein"]).loc[:, "node2_type_orig"].tolist()))
#   # print(ind)
#   ind = [i for i,r in df.iterrows() if (r["node1_type"] == "biological_process") and (r["node2_type"] == "biological_process")]
#   # display(df[ind])
#   df = df.loc[ind,:]
#   if df.empty:
#   # if df.empty:
#     print("finish")
#     print(n)
#     break
    
#   # go_treat = df.loc[:,"node2"].tolist()
#   go_treat += df.node2.tolist() 
#   # go += go_treat
#   n += 1


# GO = pd.DataFrame (go_treat, columns = ['keep_GO'])
# GO.to_csv(DIR + "res/keep_biological_functions.csv")

In [27]:
advanced = pd.read_csv("res/filtered_base_incl_bf_layer_network.csv", dtype= {"node1": str, "node2": str})
advanced

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14,15,17,20,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,tradeNames,yearOfFirstApproval,drugType,maximumClinicalTrialPhase,blackBoxWarning,ActionType_drug__filter,isApproved,withdrawnNotice,hasBeenWithdrawn,edge_type
0,151561,GO:0000002,GO:0007005,mitochondrial genome maintenance,mitochondrion organization,biological_process,biological_process,biological_process,biological_process,purple,d,purple,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,is_a
1,151562,GO:0000003,GO:0008150,reproduction,biological_process,biological_process,biological_process,biological_process,biological_process,purple,d,purple,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,is_a
2,151565,GO:0000012,GO:0006281,single strand break repair,DNA repair,biological_process,biological_process,biological_process,biological_process,purple,d,purple,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,is_a
3,151566,GO:0000017,GO:0042946,alpha-glucoside transport,glucoside transport,biological_process,biological_process,biological_process,biological_process,purple,d,purple,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,is_a
4,151567,GO:0000018,GO:0051052,regulation of DNA recombination,regulation of DNA metabolic process,biological_process,biological_process,biological_process,biological_process,purple,d,purple,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,is_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249818,151554,CHEMBL4297064,6532,MILNACIPRAN HYDROCHLORIDE,SLC6A4,drug,drug_target,drug,protein,green,s,green,o,NaN,single protein,"['Dalcipran', 'Ixel', 'Milnacipran hydrochlori...",2009.0,Small molecule,4.0,1.0,INHIBITOR,1.0,NaN,0.0,NaN
249819,151555,CHEMBL259209,6532,MILNACIPRAN,SLC6A4,drug,drug_target,drug,protein,green,s,green,o,F03,single protein,[],2009.0,Small molecule,4.0,1.0,INHIBITOR,1.0,NaN,0.0,NaN
249820,151556,CHEMBL359744,7153,DOXORUBICIN HYDROCHLORIDE,TOP2A,drug,drug_target,drug,protein,green,s,green,o,C04;C20;C15;C14,single protein,"['Adriamycin', 'Adriamycin Rdf', 'Adriamycin p...",1974.0,Small molecule,4.0,1.0,INHIBITOR,1.0,NaN,0.0,NaN
249821,151557,CHEMBL53463,7153,DOXORUBICIN,TOP2A,drug,drug_target,drug,protein,green,s,green,o,NaN,single protein,['Adriblastina'],1974.0,Small molecule,4.0,1.0,INHIBITOR,1.0,NaN,0.0,NaN


In [28]:
pp = [i for i,r in advanced.iterrows() if (r["node1_type_orig"] == "protein") and (r["node2_type_orig"] == "protein")]

In [29]:
len(pp)

132384

In [10]:
advanced.loc[pp,:]

,Unnamed: 0,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,tradeNames,yearOfFirstApproval,drugType,maximumClinicalTrialPhase,blackBoxWarning,ActionType_drug__filter,isApproved,withdrawnNotice,hasBeenWithdrawn,edge_type
98264,0,381,375,ARF5,ARF1,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98265,1,381,9727,ARF5,RAB11FIP3,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98266,2,381,2108,ARF5,ETFA,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98267,3,381,23647,ARF5,ARFIP2,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98268,4,4074,26088,M6PR,GGA1,disease_related_protein,disease_related_protein,protein,protein,red,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230643,132379,51386,6231,EIF3L,RPS26,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230644,132380,51386,6230,EIF3L,RPS25,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230645,132381,51386,6233,EIF3L,RPS27A,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230646,132382,51386,6205,EIF3L,RPS11,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
advanced.loc[pp,:].rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(advanced.loc[pp,:].rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")



node_type
disease_related_protein    5297
protein                    4822
drug_target                1103
dtype: int64

In [36]:
5297+4822+1103

11222

In [31]:
test2 = advanced.rename(columns={"node1": "node", "node1_type_orig": "node_type"}).loc[:,["node", "node_type"]].append(advanced.rename(columns={"node2": "node", "node2_type_orig": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"])
test2


,node,node_type
0,GO:0000002,biological_process
1,GO:0000003,biological_process
2,GO:0000012,biological_process
3,GO:0000017,biological_process
4,GO:0000018,biological_process
...,...,...
247458,765,protein
247463,377677,protein
247464,11238,protein
247465,763,protein


In [32]:
test3 = set([r["node"] for i,r in test2.iterrows() if r["node_type"] == "protein"])
len(test3)

14479

In [40]:
len(test3-test)

3257

In [41]:
test3-test

{'3859',
 '84627',
 '26280',
 '338339',
 '169522',
 '3126',
 '8459',
 '27255',
 '338773',
 '132864',
 '64220',
 '23457',
 '92014',
 '100616496',
 '2853',
 '152110',
 '8323',
 '8529',
 '693213',
 '25876',
 '56311',
 '55289',
 '54848',
 '163255',
 '23742',
 '55647',
 '728116',
 '100302236',
 '57570',
 '126695',
 '406990',
 '5066',
 '4504',
 '3238',
 '55186',
 '8000',
 '494327',
 '10991',
 '140880',
 '23305',
 '89839',
 '3269',
 '201501',
 '56302',
 '339976',
 '6568',
 '140',
 '3231',
 '2877',
 '153339',
 '727910',
 '494331',
 '29923',
 '84329',
 '4239',
 '27159',
 '442915',
 '222643',
 '57115',
 '9103',
 '1240',
 '4649',
 '337975',
 '79077',
 '78997',
 '3352',
 '7837',
 '406885',
 '50486',
 '8526',
 '3062',
 '64478',
 '23400',
 '81537',
 '170506',
 '255061',
 '56606',
 '5076',
 '4837',
 '9509',
 '348180',
 '574033',
 '2662',
 '54546',
 '25974',
 '151742',
 '221074',
 '200424',
 '124976',
 '1113',
 '435',
 '79087',
 '64067',
 '79709',
 '9148',
 '6916',
 '338322',
 '6620',
 '53905',
 '1520

In [51]:
ind = advanced.isin(["26280"]).loc[:,["node1", "node2"]]
advanced[ind.node1]

,Unnamed: 0,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,tradeNames,yearOfFirstApproval,drugType,maximumClinicalTrialPhase,blackBoxWarning,ActionType_drug__filter,isApproved,withdrawnNotice,hasBeenWithdrawn,edge_type
242632,144368,26280,C0006142,IL1RAPL2,Malignant neoplasm of breast,disease_related_protein,disease,protein,disease,red,o,red,^,C04;C17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
